# NVIDIA AI Data Platform: Enterprise RAG with Multimodal PDF Extraction

**The Problem:** Most RAG demos work great on clean text but *fail* when they hit real enterprise documents — PDFs packed with data tables, charts, and mixed layouts. Ask a standard RAG system "What was Q3 revenue?" and you'll get a hallucinated guess, because the table got flattened into meaningless text.

**The Solution:** [NVIDIA NVIngest](https://github.com/NVIDIA-AI-Blueprints/multimodal-pdf-data-extraction) (part of the NVIDIA AI Blueprint for RAG) extracts tables and charts as **separate, structured objects** — not just raw text. This means the LLM can actually read the table rows and give you the right number.

In this notebook we will:
1. **Ingest** a financial report PDF via the NVIngest API
2. **Inspect** the structured JSON output — proving tables and charts are treated differently from text
3. **Index** the results using LlamaIndex with NVIDIA NIMs for embeddings
4. **Chat** with the data and get a precise, table-grounded answer

---

### Prerequisites

Run these commands from the repo root before opening this notebook:

```bash
# 1. Set your NGC API key
export NGC_API_KEY="your-ngc-api-key"

# 2. Log in to the NVIDIA container registry
echo "$NGC_API_KEY" | docker login nvcr.io -u '$oauthtoken' --password-stdin

# 3. Start Milvus (vector database)
bash setup.sh    # NVIngest errors at the end are expected — we start it below

# 4. Start NVIngest (document extraction service)
docker compose -f docker-compose-nvingest.yaml up -d

# 5. Wait for NVIngest to be ready (takes ~60s on first start)
until curl -sf http://localhost:7670/v1/health/ready; do sleep 5; echo "waiting..."; done

# 6. Create a Python venv and install dependencies
python3 -m venv .venv && source .venv/bin/activate
pip install -r requirements.txt && pip install "setuptools<72"

# 7. Generate the sample PDF
python generate_sample_pdf.py

# 8. Launch this notebook
jupyter lab notebooks/
```

**To tear down** when finished:
```bash
docker compose -f docker-compose-nvingest.yaml down -v
bash teardown.sh
```

## Part 0 — Configuration & Setup

In [ ]:
import os
import json
import base64
from pathlib import Path
from pprint import pprint
from collections import Counter

import requests
import pandas as pd

# ── Service endpoints ────────────────────────────────────────────────────────
# NVIngest runtime listens on port 7670 (standalone deployment)
NVINGEST_URL = os.getenv("NVINGEST_URL", "http://localhost:7670")
MILVUS_URI   = os.getenv("MILVUS_URI",   "http://localhost:19530")

# ── NVIDIA API Key (NGC_API_KEY or NVIDIA_API_KEY) ───────────────────────────
NVIDIA_API_KEY = os.getenv("NGC_API_KEY") or os.getenv("NVIDIA_API_KEY", "")
assert NVIDIA_API_KEY, (
    "Set NGC_API_KEY or NVIDIA_API_KEY in your environment.  "
    "Get one at https://build.nvidia.com/"
)

# ── Path to the sample PDF ────────────────────────────────────────────────────
PDF_PATH = Path("../data/sample_financial_report.pdf")
assert PDF_PATH.exists(), f"PDF not found at {PDF_PATH}. Run: python generate_sample_pdf.py"

print(f"NVIngest endpoint : {NVINGEST_URL}")
print(f"Milvus endpoint   : {MILVUS_URI}")
print(f"NVIDIA API Key    : {NVIDIA_API_KEY[:12]}...")
print(f"Sample PDF        : {PDF_PATH}  ({PDF_PATH.stat().st_size / 1024:.1f} KB)")

In [ ]:
# Quick health check on NVIngest
try:
    resp = requests.get(f"{NVINGEST_URL}/v1/health/ready", timeout=10)
    print(f"NVIngest health: {resp.status_code} — {resp.text[:200]}")
except Exception as e:
    print(f"⚠️  Could not reach NVIngest at {NVINGEST_URL}: {e}")
    print("   Make sure NVIngest is running. Check: docker ps")

---

## Part 1 — Ingest: Send the PDF to NVIngest

NVIngest provides a REST API that accepts PDF documents and returns structured
extractions.  Unlike a simple text parser (PyPDF2, pdfplumber), NVIngest uses
GPU-accelerated models to:

- **Detect tables** via YOLOX object detection
- **Parse table structure** via DePlot / table-transformer
- **OCR** any scanned content via PaddleOCR
- **Caption charts** via a Vision-Language Model

The result: each element is tagged with its *type* (`text`, `table`, `image`)
so downstream systems can treat them appropriately.

In [ ]:
# ── Approach A: Using the nv-ingest-client library ────────────────────────────
#
# The official Python client wraps the REST API with a convenient job-based
# interface.  If you have `nv-ingest-client` installed, this is the
# recommended path.

USE_CLIENT_LIB = True  # Set to False to use the raw REST fallback below

nvingest_results = None

if USE_CLIENT_LIB:
    try:
        from nv_ingest_client.client import NvIngestClient
        from nv_ingest_api.util.service_clients.rest.rest_client import RestClient
        from nv_ingest_client.primitives.tasks import (
            ExtractTask,
            SplitTask,
        )
        from nv_ingest_client.primitives.jobs import JobSpec
        from nv_ingest_client.util.file_processing.extract import extract_file_content

        print("✅ nv-ingest-client library found (v25.4.2+).")

        # Create the client pointing at our local NVIngest server (port 7670)
        client = NvIngestClient(
            message_client_allocator=RestClient,
            message_client_hostname="localhost",
            message_client_port=7670,
        )

        # Read and encode the PDF
        file_content, file_type = extract_file_content(str(PDF_PATH))

        # Build a job specification
        job_spec = JobSpec(
            payload=file_content,
            source_id=str(PDF_PATH),
            source_name=PDF_PATH.name,
            document_type=file_type,
        )

        # Add extraction task — extract text, tables, and charts
        extract_task = ExtractTask(
            document_type=file_type,
            extract_text=True,
            extract_images=True,
            extract_tables=True,
        )
        job_spec.add_task(extract_task)

        # Optional: add a splitting/chunking task
        split_task = SplitTask(
            chunk_size=1024,
            chunk_overlap=150,
        )
        job_spec.add_task(split_task)

        # Submit the job
        job_id = client.add_job(job_spec)
        client.submit_job(job_id, job_queue_id="morpheus_task_queue")

        # Wait for results
        result = client.fetch_job_result(job_id, timeout=120)
        nvingest_results = result

        print(f"✅ Ingestion complete. Received {len(result)} result set(s).")

    except ImportError:
        print("⚠️  nv-ingest-client not installed. Falling back to REST API...")
        USE_CLIENT_LIB = False
    except Exception as e:
        print(f"⚠️  Client library call failed: {e}")
        print("   Falling back to REST API approach...")
        USE_CLIENT_LIB = False

In [ ]:
# ── Approach B: Raw REST API fallback ─────────────────────────────────────────
#
# If the nv-ingest-client is not available or the API has changed, we can
# fall back to the NV-Ingest v2 REST API (submit_job + fetch_job).

if not USE_CLIENT_LIB or nvingest_results is None:
    print("Using REST API to ingest the PDF...")

    # Read and base64-encode the PDF
    pdf_bytes = PDF_PATH.read_bytes()
    pdf_b64 = base64.b64encode(pdf_bytes).decode("utf-8")

    # Build a v2 submit_job payload
    payload = {
        "payload": pdf_b64,
        "source_id": str(PDF_PATH),
        "source_name": PDF_PATH.name,
        "document_type": "pdf",
        "tasks": [
            {
                "type": "extract",
                "task_properties": {
                    "method": "pdfium",
                    "document_type": "pdf",
                    "extract_text": True,
                    "extract_images": True,
                    "extract_tables": True,
                }
            },
            {
                "type": "split",
                "task_properties": {
                    "chunk_size": 1024,
                    "chunk_overlap": 150,
                }
            }
        ]
    }

    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
    }

    # Submit the job
    resp = requests.post(
        f"{NVINGEST_URL}/v2/submit_job",
        json=payload,
        headers=headers,
        timeout=300,
    )
    resp.raise_for_status()
    submit_result = resp.json()
    job_id = submit_result  # v2 returns the job_id directly
    print(f"Job submitted: {job_id}")

    # Fetch results (poll until ready)
    import time
    for attempt in range(60):
        fetch_resp = requests.get(
            f"{NVINGEST_URL}/v2/fetch_job/{job_id}",
            headers=headers,
            timeout=60,
        )
        if fetch_resp.status_code == 200:
            nvingest_results = fetch_resp.json()
            print(f"✅ Ingestion complete (REST v2). Status: {fetch_resp.status_code}")
            break
        elif fetch_resp.status_code == 202:
            print(f"  Still processing... (attempt {attempt+1})")
            time.sleep(2)
        else:
            fetch_resp.raise_for_status()
    else:
        print("⚠️  Timed out waiting for ingestion result.")

---

## Part 2 — Inspect: Explore the Structured Extraction Output

This is the key insight: NVIngest does **not** return a flat wall of text.
Instead, each element is classified by `type`:

| Type    | What it means                                    |
|---------|--------------------------------------------------|
| `text`  | Narrative paragraphs, headings                   |
| `table` | A detected table with row/column structure        |
| `image` | A chart, figure, or infographic                  |

Let's look at what came back.

In [ ]:
# ── Parse and normalise the results into a flat list of chunks ─────────────
#
# The exact shape of `nvingest_results` depends on the NVIngest version.
# We normalise it into a list of dicts, each with at least:
#   { "type": "text"|"table"|"image", "content": ..., "metadata": {...} }

def flatten_results(raw):
    """Extract a flat list of content chunks from NVIngest output.

    NVIngest v25+ returns results as a list of list-of-dicts.
    Each dict has 'document_type' and 'metadata' with nested content.
    For structured items (tables/charts), the actual text is in
    metadata.table_metadata.table_content, NOT in metadata.content
    (which is a base64-encoded image).
    """
    chunks = []

    # Handle different response shapes
    if isinstance(raw, list):
        items = raw
    elif isinstance(raw, dict):
        items = raw.get("data", raw.get("documents", raw.get("results", [raw])))
    else:
        items = [raw]

    for item in items:
        if isinstance(item, list):
            # Nested list — recurse
            chunks.extend(flatten_results(item))
        elif isinstance(item, dict):
            meta = item.get("metadata", {})
            content_metadata = meta.get("content_metadata", {})

            # Determine content type
            content_type = (
                content_metadata.get("type")
                or item.get("document_type")
                or item.get("type")
                or "text"
            )

            # Extract the actual text content based on type
            # Note: metadata fields may be None, so use `or {}` to handle that
            table_meta = meta.get("table_metadata") or {}
            chart_meta = meta.get("chart_metadata") or {}
            description = content_metadata.get("description") or ""

            if content_type == "structured":
                # For structured items (tables/charts), get table_content text
                table_text = table_meta.get("table_content") or ""
                # Classify as table or chart based on description
                if "chart" in description.lower() or "figure" in description.lower():
                    content_type = "image"  # chart/figure
                else:
                    content_type = "table"
                content = table_text if table_text else (meta.get("content") or "")
            elif content_type == "text":
                content = (meta.get("content") or "") or (item.get("content") or "")
            elif content_type == "image":
                # For images, check if there's a caption or table_content
                content = (
                    (table_meta.get("table_content") or "")
                    or (chart_meta.get("caption") or "")
                    or description
                    or "Image from document"
                )
                if not content or content == "Image extracted from PDF document.":
                    content = "(image data)"
            else:
                content = (meta.get("content") or "") or (item.get("content") or "")

            # Map remaining structured types
            if content_type in ("table_data",):
                content_type = "table"
            if content_type in ("chart", "figure", "infographic"):
                content_type = "image"

            chunks.append({
                "type": content_type,
                "content": content if isinstance(content, str) else json.dumps(content, indent=2) if content else "",
                "metadata": meta,
            })
        elif isinstance(item, str):
            chunks.append({"type": "text", "content": item, "metadata": {}})

    return chunks


chunks = flatten_results(nvingest_results)
print(f"Total chunks extracted: {len(chunks)}")
print()

# Summary by type
type_counts = Counter(c["type"] for c in chunks)
print("Chunk types:")
for t, count in type_counts.most_common():
    print(f"  {t:10s}  →  {count}")

In [ ]:
# ── Show ALL chunks (truncated for readability) ──────────────────────────────
print("=" * 80)
print("FULL EXTRACTION OUTPUT")
print("=" * 80)

for i, chunk in enumerate(chunks):
    content_preview = chunk["content"][:300] if chunk["content"] else "(empty)"
    print(f"\n─── Chunk {i}  │  type: {chunk['type']}  ───")
    print(content_preview)
    if len(chunk["content"]) > 300:
        print(f"  ... ({len(chunk['content'])} chars total)")

In [ ]:
# ── Highlight TABLE chunks ───────────────────────────────────────────────────
table_chunks = [c for c in chunks if c["type"] == "table"]

print(f"\n{'='*80}")
print(f"🔍 TABLE CHUNKS FOUND: {len(table_chunks)}")
print(f"{'='*80}")

if table_chunks:
    for i, tc in enumerate(table_chunks):
        print(f"\n── Table {i + 1} ──")
        print(tc["content"][:1000])
        print()
else:
    print("\n(No table chunks detected. Check NVIngest configuration.)")
    print("Tip: Tables might be classified under a different type key.")
    print("Let's check all unique types:", set(c['type'] for c in chunks))

In [ ]:
# ── Highlight IMAGE / CHART chunks ───────────────────────────────────────────
image_chunks = [c for c in chunks if c["type"] == "image"]

print(f"\n{'='*80}")
print(f"🖼️  IMAGE / CHART CHUNKS FOUND: {len(image_chunks)}")
print(f"{'='*80}")

if image_chunks:
    for i, ic in enumerate(image_chunks):
        print(f"\n── Image {i + 1} ──")
        # Images may be base64-encoded; show metadata instead of raw data
        meta = ic.get("metadata", {})
        content_preview = ic["content"][:200] if ic["content"] else "(base64 data)"
        print(f"  Content preview: {content_preview}")
        if meta:
            print(f"  Metadata keys:   {list(meta.keys())}")
else:
    print("\n(No image/chart chunks detected.)")

### What Just Happened?

Notice how NVIngest **separated** the document into distinct content types:

- **Text chunks** contain the narrative paragraphs (executive summary, guidance, etc.)
- **Table chunks** contain the financial data table — with the quarterly revenue, net income, and EPS values preserved as structured data
- **Image chunks** contain the bar chart figure

A standard text parser (e.g., `PyPDF2` or `pdfplumber`) would have flattened the table into something like:

```
Quarter Revenue ($M) Net Income ($M) EPS ($) Q1 2024 2,105 312 1.56 Q2 2024 2,498 389 1.95 ...
```

That mangled string would be nearly impossible for an LLM to parse accurately.
NVIngest's structured extraction preserves the table's **row-column relationships**, which is what makes precise Q&A possible.

---

## Part 3 — Index: Build the RAG Pipeline with LlamaIndex & NVIDIA NIMs

Now we'll take these structured chunks and build a proper retrieval-augmented
generation pipeline using:

- **`llama-index-llms-nvidia`** — connects to NVIDIA-hosted LLM (meta/llama-3.1-70b-instruct)
- **`llama-index-embeddings-nvidia`** — connects to NVIDIA-hosted embedding model
- **`llama-index-vector-stores-milvus`** — stores vectors in our local Milvus instance

In [ ]:
from llama_index.core import VectorStoreIndex, Document, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.nvidia import NVIDIA
from llama_index.embeddings.nvidia import NVIDIAEmbedding

# ── Configure NVIDIA LLM ──────────────────────────────────────────────────────
llm = NVIDIA(
    model="meta/llama-3.1-70b-instruct",
    api_key=NVIDIA_API_KEY,
    base_url="https://integrate.api.nvidia.com/v1",
)

# ── Configure NVIDIA Embedding Model ──────────────────────────────────────────
embed_model = NVIDIAEmbedding(
    model="nvidia/nv-embedqa-e5-v5",
    api_key=NVIDIA_API_KEY,
    base_url="https://integrate.api.nvidia.com/v1",
)

# ── Set as global defaults ────────────────────────────────────────────────────
Settings.llm = llm
Settings.embed_model = embed_model

print(f"LLM:        {llm.model}")
print(f"Embeddings: {embed_model.model}")

In [ ]:
# ── Convert NVIngest chunks into LlamaIndex Documents ────────────────────────
#
# We preserve the content type as metadata so the retriever can distinguish
# between text paragraphs and structured table data.

documents = []
for i, chunk in enumerate(chunks):
    content = chunk["content"]
    if not content or not content.strip():
        continue

    # Skip placeholder image chunks that have no real text content
    if content.strip() == "(image data)":
        continue

    # For table chunks, prefix with a label to help the LLM identify context
    if chunk["type"] == "table":
        content = f"[TABLE DATA]\n{content}"
    elif chunk["type"] == "image":
        content = f"[CHART/FIGURE]\n{content}"

    doc = Document(
        text=content,
        metadata={
            "source": "sample_financial_report.pdf",
            "content_type": chunk["type"],
            "chunk_index": i,
        },
    )
    documents.append(doc)

print(f"Created {len(documents)} LlamaIndex documents:")
for doc in documents:
    ct = doc.metadata["content_type"]
    preview = doc.text[:80].replace("\n", " ")
    print(f"  [{ct:6s}] {preview}...")

In [ ]:
# ── Build the Vector Index ───────────────────────────────────────────────────
#
# Option A: Use Milvus vector store (recommended for production)
# Option B: Use in-memory index (simpler, good for demos)

USE_MILVUS = True  # Set to False for a simpler in-memory index

if USE_MILVUS:
    try:
        from llama_index.vector_stores.milvus import MilvusVectorStore
        from llama_index.core import StorageContext

        vector_store = MilvusVectorStore(
            uri=MILVUS_URI,
            collection_name="nvidia_rag_demo",
            dim=1024,  # nv-embedqa-e5-v5 outputs 1024-dim vectors
            overwrite=True,  # Fresh index each run
        )

        storage_context = StorageContext.from_defaults(vector_store=vector_store)

        index = VectorStoreIndex.from_documents(
            documents,
            storage_context=storage_context,
            show_progress=True,
        )
        print("\n✅ Vector index built with Milvus backend.")

    except Exception as e:
        print(f"⚠️  Milvus connection failed: {e}")
        print("   Falling back to in-memory index...")
        USE_MILVUS = False

if not USE_MILVUS:
    index = VectorStoreIndex.from_documents(
        documents,
        show_progress=True,
    )
    print("\n✅ Vector index built (in-memory).")

In [ ]:
# ── Create the Chat Engine ───────────────────────────────────────────────────
#
# We set similarity_top_k high enough to always retrieve the table chunk
# alongside text — this is critical for table-grounded Q&A.

chat_engine = index.as_chat_engine(
    chat_mode="condense_plus_context",
    similarity_top_k=10,
    verbose=False,
    system_prompt=(
        "You are a helpful financial analyst assistant. You have access to "
        "Acme Corp's quarterly earnings data including revenue tables, "
        "financial charts, and narrative text. When answering questions about "
        "specific numbers, always cite the exact figures from the data. "
        "If the data includes a table, read the specific row and column to "
        "find the answer."
    ),
)

print("✅ Chat engine ready.")

---

## Part 4 — The "Aha!" Moment: Precise Table-Grounded Q&A

Now for the payoff. Let's ask questions that **require reading specific rows in a table** — the exact scenario where standard RAG fails.

Remember the ground-truth data in our financial report:

| Quarter  | Revenue ($M) | Net Income ($M) | EPS ($) |
|----------|-------------|-----------------|--------|
| Q1 2024  | 2,105       | 312             | 1.56   |
| Q2 2024  | 2,498       | 389             | 1.95   |
| Q3 2024  | 2,847       | 456             | 2.28   |
| Q4 2024  | 3,102       | 521             | 2.61   |

In [ ]:
# ── Query 1: The headline question ───────────────────────────────────────────

print("=" * 80)
print("QUERY 1: What was Acme Corp's revenue in Q3 2024?")
print("=" * 80)

response1 = chat_engine.chat("What was Acme Corp's revenue in Q3 2024?")

print(f"\n💡 Answer:\n{response1.response}")
print(f"\n📎 Expected: $2,847M (from the table, Q3 2024 row, Revenue column)")

In [ ]:
# ── Query 2: Requires comparing two rows ─────────────────────────────────────

print("=" * 80)
print("QUERY 2: Compare Q2 and Q3 net income. What was the growth?")
print("=" * 80)

response2 = chat_engine.chat(
    "Compare Acme Corp's net income between Q2 2024 and Q3 2024. "
    "What was the dollar increase and percentage growth?"
)

print(f"\n💡 Answer:\n{response2.response}")
print(f"\n📎 Expected: Q2=$389M → Q3=$456M, increase of $67M (~17.2% growth)")

In [ ]:
# ── Query 3: EPS — the most granular table lookup ────────────────────────────

print("=" * 80)
print("QUERY 3: What was the EPS in Q4 2024?")
print("=" * 80)

response3 = chat_engine.chat("What was Acme Corp's earnings per share (EPS) in Q4 2024?")

print(f"\n💡 Answer:\n{response3.response}")
print(f"\n📎 Expected: $2.61 (from the table, Q4 2024 row, EPS column)")

In [ ]:
# ── Query 4: Requires understanding the full table + narrative ────────────────

print("=" * 80)
print("QUERY 4: What was the total annual revenue and what is the guidance?")
print("=" * 80)

response4 = chat_engine.chat(
    "What was Acme Corp's total annual revenue for FY 2024, "
    "and what is the company's revenue guidance for Q1 2025?"
)

print(f"\n💡 Answer:\n{response4.response}")
print(f"\n📎 Expected: FY 2024 total = $10,552M; Q1 2025 guidance = $3,300-$3,500M")

---

## Conclusion

### What We Demonstrated

1. **NVIngest** parsed a PDF financial report and separated its content into distinct types: narrative *text*, structured *tables*, and *charts*.

2. **The table was preserved as structured data** — not flattened into an unreadable string. This is the critical difference between NVIDIA's approach and a naive text parser.

3. **LlamaIndex + NVIDIA NIMs** vectorised the structured chunks and created a chat engine capable of answering precise questions.

4. **The LLM correctly answered table-specific questions** ("What was Q3 revenue?" → "$2,847M") because it received the actual table structure, not garbled text.

### Why This Matters for Enterprise RAG

Real enterprise documents — financial reports, contracts, research papers — are **not** plain text. They contain:
- Data tables with hundreds of rows
- Charts and figures with axis labels
- Multi-column layouts and footnotes

Standard text extraction destroys this structure. NVIDIA's AI Data Platform preserves it, making enterprise RAG **actually work** on the documents that matter most.

### Architecture Recap

```
PDF Document
  │
  ▼
NVIngest (GPU-accelerated extraction)
  │  ├── text chunks
  │  ├── table chunks  ← structured rows & columns
  │  └── image chunks  ← chart descriptions
  ▼
NVIDIA Embedding NIM  →  Milvus Vector DB
  │
  ▼
NVIDIA LLM NIM  →  Accurate, grounded answers
```

### Next Steps

- Try with your own enterprise PDFs (annual reports, contracts, etc.)
- Enable the Vision Language Model for richer chart understanding
- Explore hybrid search (dense + sparse) for even better retrieval
- Add evaluation with RAGAS to measure answer quality

---

*Built with the [NVIDIA AI Blueprint for RAG](https://build.nvidia.com/nvidia/build-an-enterprise-rag-pipeline) and [NVIngest](https://github.com/NVIDIA-AI-Blueprints/multimodal-pdf-data-extraction).*